In [1]:
import numpy as np
import pandas as pd
from pyproj import Transformer
from datetime import date,datetime

In [144]:
#Import general library
import bokeh

#Import to handle data parsed to bokeh plots
from bokeh.models import ColumnDataSource

#Import to handle bokeh notebook
from bokeh.io import output_notebook

#Import for the background of the map plot
from bokeh.tile_providers import CARTODBPOSITRON, get_provider, CARTODBPOSITRON_RETINA, STAMEN_TERRAIN, STAMEN_TONER, WIKIMEDIA

#Import to create plot, show plot, and output plot
from bokeh.plotting import figure, output_file, show

#Import to handle layout of the bokeh plots
from bokeh.layouts import column, row, layout

#Importing the RangeTool used in the Timeline plot
from bokeh.models import RangeTool

#Import to set the range of the Timeline
from bokeh.models import Range1d

#Import for the button and datepicker widget
from bokeh.models import Button, DatePicker

#Import for callback on an event
from bokeh.events import ButtonClick

#Import palette for the color mapper used in geo_map
#Possible other color palettes
# https://colorcet.holoviz.org/user_guide/index.html
import colorcet as cc
#palette = cc.CET_L9

#Import palette for the histogram colors
from bokeh.palettes import Spectral

palette = Spectral


#Import log_cmap to support the color mapper for the geo_map
from bokeh.transform import log_cmap, linear_cmap

#Importing the select tools for bokeh plots
from bokeh.models import LassoSelectTool, BoxSelectTool

#Import the tools for the color bar
from bokeh.models import ColorBar, LogTicker
#Other option for ticker: AdaptiveTicker

#Import legend to show the legend for the timeline and histogram
from bokeh.models import Legend

#Import for the histogram model
from bokeh.models import FactorRange

#Import for the coloring of the bars in the histogram plot
from bokeh.transform import factor_cmap

#Import to show multiple tabs
from bokeh.models import Panel, Tabs, CategoricalColorMapper

output_notebook()

Loading BokehJS ...

## Assumptions

In [4]:
#screen resolution
screen_height = 680
screen_length = 920

#Location of dataset csv
location_twitter_dataset = "D:/Master Project/complete_swiss_dataset.csv"

location_weather_data = "D:/Master Project/Extracted Radar Data/"

## Data retrieval

In [336]:
#Function to retrieve the csv storing the weather data
def retrieve_weather_data(measurement, year, day):
    
    #Check whether the parameters passed into the function are valid
    if(len(measurement) == 0 or year < 0 or day < 0 or day > 370):
        print("Error parameters incorrect retrieve_weather_data")
        return -1
    
    weather_data = pd.read_csv(location_weather_data+measurement+str(year)+str(day)+".csv")
    del weather_data['Unnamed: 0']
    
    times = np.unique(weather_data['timestamp'])
    
    times_dict = {}

    for time in times:
        times_dict[time] = datetime.fromtimestamp(time)
        
    weather_data['datetime'] = weather_data['timestamp'].map(times_dict)
    
    weather_data['hour_minute'] = weather_data['datetime'].dt.strftime('%H:%M')
    
    weather_data['value'] = weather_data['value'].astype(str)
    
    #transformer_merc = Transformer.from_crs("EPSG:4326", "EPSG:3857") (for verification)
    #np.array(transformer_merc.transform([45.8,47.9,45.8,47.9],[5.9,5.9,10.6,10.6])).T (for verification)
    
    #Filter based on the coordinates 45.8, 5.9 and 47.9,10.6 which encompass switzerland
    #Translated to web mercator
    geo_filter = np.where((weather_data['x_mercator'] <= 1170000) &
                          (weather_data['x_mercator'] >= 657000) & 
                          (weather_data['y_mercator'] <= 6090000) & 
                          (weather_data['y_mercator'] >= 5750000))
    
    return weather_data.iloc[geo_filter]

In [337]:
BZC = retrieve_weather_data("BZC",19,242)

In [291]:
BZC = retrieve_weather_data("BZC",19,242)
CZC = retrieve_weather_data("CZC",19,242)
LZC = retrieve_weather_data("LZC",19,242)
RZC = retrieve_weather_data("RZC",19,242)

In [328]:
len(np.where(BZC['y_mercator'] <= 6090000)[0])

23951

In [329]:
len(BZC['y_mercator'])

23951

In [279]:
BZC.iloc[0]['timestamp']

1567157100.0

In [251]:
times = np.unique(BZC['timestamp'])

In [252]:
times_dict = {}

for date in dates:
    times_dict[date] = datetime.fromtimestamp(date)

In [253]:
BZC['datetime'] = BZC['timestamp'].map(times_dict)

In [254]:
BZC['hour_minute'] = BZC['datetime'].dt.strftime('%H:%M')

In [101]:
datetime.fromtimestamp(1567157100.0)

datetime.datetime(2019, 8, 30, 11, 25)

In [102]:
datetime.timestamp

<method 'timestamp' of 'datetime.datetime' objects>

In [343]:
#Define the weather app
def W_app(doc):

    #load the background of the map plot
    tile_provider = get_provider(CARTODBPOSITRON_RETINA)

    #Create the Map figure
    # range bounds supplied in web mercator coordinates
    weather_map = figure(x_range=(650000, 1180000), y_range=(5700000, 6100000),
                         x_axis_type="mercator", y_axis_type="mercator",
                         plot_width=screen_length, plot_height=screen_height)
    
    weather_map.add_tools(LassoSelectTool())
    
    #Add the background to the map
    weather_map.add_tile(tile_provider)
    
    weather_data = BZC #.iloc[np.where(BZC['hour_minute'] == '15:20')]
    
    #Load in the map data
    source_weather = ColumnDataSource(weather_data)
    
    #Initialize the color mapper based on the values of density in the map_data

    color_mapper = CategoricalColorMapper(palette=Spectral[11], factors=np.unique(BZC['value'].astype(str)))
    #color_mapper = log_cmap(field_name = 'density', palette = palette,
    #                           low = map_data['density'].min(),
    #                           high = map_data['density'].max())
    
    #Defines color bar which indicates the levels
    #color_bar = ColorBar(color_mapper=color_mapper['transform'], 
    #                 ticker = LogTicker(num_minor_ticks=1), 
    #                 label_standoff = 13, width=20, location=(0,0))# Set color_bar location
    
    #Add the color bar to the figure
    #weather_map.add_layout(color_bar, 'left')
    
    #Plot the data points with their color corresponding to the amount of tweets at the location
    #geo_map.circle(x='x_mercator', y='y_mercator', size=2, color=color_mapper, alpha=1, source=source_map)
    
    weather_map.rect(x='x_mercator', y='y_mercator',
                     width=1450, height=1450, #As big as possible without overlap
                     alpha=1, source=source_weather,
                    fill_color=dict(field='value',transform=color_mapper),
                    line_alpha=0)
    
    

    plots = layout([
        [weather_map],
        
    ])
    doc.add_root(plots)



In [344]:
show(W_app)

In [156]:
np.unique(BZC['value']).astype(str)

array(['2.0', '11.0', '21.0', '31.0', '41.0', '51.0', '61.0', '71.0',
       '81.0', '91.0'], dtype='<U32')

In [256]:
BZC['value'] = BZC['value'].astype(str)

In [261]:
type(BZC.iloc[0]['hour_minute'])

str

In [262]:
time_periods = np.unique(BZC['hour_minute'])

In [271]:
np.where(BZC['timestamp'] == 1567153800)

(array([], dtype=int64),)